In [ ]:
from sklearn.cluster import KMeans 
from sklearn.preprocessing import StandardScaler
import numpy as np
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.decomposition import PCA
import pandas as pd

In [ ]:
# load data
session1 = pd.read_csv("/content/drive/Shareddrives/207/cluster/time_reduced_session_agg1.csv")
session2 = pd.read_csv("/content/drive/Shareddrives/207/cluster/time_reduced_session_agg2.csv")
session3 = pd.read_csv("/content/drive/Shareddrives/207/cluster/time_reduced_session_agg3.csv")
session4 = pd.read_csv("/content/drive/Shareddrives/207/cluster/time_reduced_session_agg4.csv")
session5 = pd.read_csv("/content/drive/Shareddrives/207/cluster/time_reduced_session_agg5.csv")

In [ ]:
# drop the redundant columns and add a new row for session ids
session1 = session1.drop(['Unnamed: 0', 'type', 'Group.2'], axis=1)
session1.loc[len(session1.index)] = ['session_id', 'session_id'] + (session1.shape[1]-2) * ['1'] 
session2 = session2.drop(['Unnamed: 0', 'type', 'Group.2'], axis=1)
session2.loc[len(session2.index)] = ['session_id', 'session_id'] + (session2.shape[1]-2) * ['2'] 
session3 = session3.drop(['Unnamed: 0', 'type', 'Group.2'], axis=1)
session3.loc[len(session3.index)] = ['session_id', 'session_id'] + (session3.shape[1]-2) * ['3'] 
session4 = session4.drop(['Unnamed: 0', 'type', 'Group.2'], axis=1)
session4.loc[len(session4.index)] = ['session_id', 'session_id'] + (session4.shape[1]-2) * ['4'] 
session5 = session5.drop(['Unnamed: 0', 'type', 'Group.2'], axis=1)
session5.loc[len(session5.index)] = ['session_id', 'session_id'] + (session5.shape[1]-2) * ['5'] 

In [ ]:
# merge sessions with different contrasts 
sessions = pd.merge(session1, session2, on=["V1", "V2"])
sessions = pd.merge(sessions, session3, on=["V1", "V2"])
sessions = pd.merge(sessions, session4, on=["V1", "V2"])
sessions = pd.merge(sessions, session5, on=["V1", "V2"])

<ipython-input-55-2443ebc95386>:4: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'V54_x', 'V47_x', 'V103_x', 'V112_x', 'V17_x', 'V24_x', 'V41_x', 'V67_x', 'V53_x', 'V98_x', 'V70_x', 'V30_x', 'V87_x', 'V7_x', 'V18_x', 'V118_x', 'V26_x', 'V109_x', 'V116_x', 'V121_x', 'V94_x', 'V3_x', 'V76_x', 'V23_x', 'V111_x', 'V5_x', 'V38_x', 'V66_x', 'V83_x', 'V51_x', 'V11_x', 'V8_x', 'V48_x', 'V33_x', 'V61_x', 'V62_x', 'V101_x', 'V92_x', 'V96_x', 'V100_x', 'V58_x', 'V79_x', 'V52_x', 'V50_x', 'V25_x', 'V106_x', 'V31_x', 'V110_x', 'V36_x', 'V40_x', 'V49_x', 'V13_x', 'V57_x', 'V81_x', 'V78_x', 'V85_x', 'V90_x', 'V46_x', 'V65_x', 'V114_x', 'V56_x', 'V4_x', 'V74_x', 'V44_x', 'V35_x', 'V20_x', 'V32_x', 'V117_x', 'V28_x', 'V99_x', 'V119_x', 'V88_x', 'V80_x', 'V60_x', 'V21_x', 'V69_x', 'V72_x', 'V89_x', 'V86_x', 'V34_x', 'V95_x', 'V91_x', 'V6_x', 'V73_x', 'V10_x', 'V19_x', 'V59_x', 'V113_x', 'V15_x', 'V105_x', 'V22_x', 'V108_x', 'V14_x', 'V122_x', 'V42_x', 'V93_x', 'V115_x', 'V64_x', 'V12

In [ ]:
sessions.T.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,1160.0,1160.0,1160.0,1160.0,1160.0,1160.0,1160.0,1160.0,1160.0,1160.0,1160.0,1160.0,1160.0,1160.0,1160.0,1160.0,1160
unique,546.0,89.0,290.0,283.0,166.0,121.0,143.0,263.0,291.0,98.0,135.0,96.0,332.0,263.0,136.0,129.0,6
top,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
freq,69.0,384.0,160.0,176.0,243.0,314.0,375.0,182.0,155.0,300.0,268.0,335.0,110.0,187.0,328.0,272.0,533


In [ ]:
sessions.shape

(17, 1160)

In [ ]:
# sanity checks
assert sessions.shape[0] == session1.shape[0]
assert sessions.shape[1] == session1.shape[1] + session2.shape[1] + session3.shape[1] + session4.shape[1] + session5.shape[1] - 8

In [ ]:
# convert to numpy and extract training data for clustering
x_train = sessions.to_numpy()[:-1,2:]
x_train = x_train.T
x_train.shape

(1158, 16)

In [ ]:
# Kmeans clustering when n_clusters=2
scaler = StandardScaler()
x_df = pd.DataFrame(scaler.fit_transform(x_train))
kmeans = KMeans(n_clusters=2)
kmeans.fit(x_df)
clusters = kmeans.predict(x_df)
x_df["Cluster"] = clusters

# use PCA and plot clustering results in 2d
plotdf = pd.DataFrame(np.array(x_df))
plotdf.columns = x_df.columns
pca = PCA(n_components=2)
pcadf = pd.DataFrame(pca.fit_transform(plotdf.drop(["Cluster"], axis=1)))
pcadf.columns = ["PC1_2d", "PC2_2d"]
plotdf = pd.concat([plotdf, pcadf], axis=1, join='inner')
cluster0 = plotdf[plotdf["Cluster"] == 0]
cluster1 = plotdf[plotdf["Cluster"] == 1]
init_notebook_mode(connected=True)
trace1 = go.Scatter(x = cluster0["PC1_2d"], y = cluster0["PC2_2d"], mode = "markers", name = "component 0", marker = dict(color = 'rgba(255, 128, 255, 0.8)'), text = None)
trace2 = go.Scatter(x = cluster1["PC1_2d"], y = cluster1["PC2_2d"], mode = "markers", name = "component 1", marker = dict(color = 'rgba(255, 128, 2, 0.8)'), text = None)
title = "Visualizing Kmean clustering using 2D PCA"
fig = dict(data = [trace1, trace2], layout = dict(title = title, xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False), yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)))
figure = go.Figure(fig)
figure.show(renderer="colab")

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



not clustering well.

In [ ]:
# Kmeans clustering
scaler = StandardScaler()
x_df = pd.DataFrame(scaler.fit_transform(x_train))
kmeans = KMeans(n_clusters=4)
kmeans.fit(x_df)
clusters = kmeans.predict(x_df)
x_df["Cluster"] = clusters

# use PCA and plot clustering results in 2d
plotdf = pd.DataFrame(np.array(x_df))
plotdf.columns = x_df.columns
pca = PCA(n_components=2)
pcadf = pd.DataFrame(pca.fit_transform(plotdf.drop(["Cluster"], axis=1)))
pcadf.columns = ["PC1_2d", "PC2_2d"]
plotdf = pd.concat([plotdf, pcadf], axis=1, join='inner')
cluster0 = plotdf[plotdf["Cluster"] == 0]
cluster1 = plotdf[plotdf["Cluster"] == 1]
cluster2 = plotdf[plotdf["Cluster"] == 2]
cluster3 = plotdf[plotdf["Cluster"] == 3]
init_notebook_mode(connected=True)
trace1 = go.Scatter(x = cluster0["PC1_2d"], y = cluster0["PC2_2d"], mode = "markers", name = "component 0", marker = dict(color = 'rgba(255, 128, 255, 0.8)'), text = None)
trace2 = go.Scatter(x = cluster1["PC1_2d"], y = cluster1["PC2_2d"], mode = "markers", name = "component 1", marker = dict(color = 'rgba(255, 128, 2, 0.8)'), text = None)
trace3 = go.Scatter(x = cluster2["PC1_2d"], y = cluster2["PC2_2d"], mode = "markers", name = "component 2", marker = dict(color = 'rgba(0, 200, 200, 0.8)'), text = None)
trace4 = go.Scatter(x = cluster3["PC1_2d"], y = cluster3["PC2_2d"], mode = "markers", name = "component 3", marker = dict(color = 'rgba(200, 100, 100, 0.8)'), text = None)
title = "Visualizing Kmean clustering using 2D PCA"
fig = dict(data = [trace1, trace2, trace3, trace4], layout = dict(title = title, xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False), yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)))
figure = go.Figure(fig)
figure.show(renderer="colab")

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



looks like the sparse parts are not well clustered. \\
decide to use n_clusters=3.

In [ ]:
# Kmeans clustering
scaler = StandardScaler()
x_df = pd.DataFrame(scaler.fit_transform(x_train))
kmeans = KMeans(n_clusters=3)
kmeans.fit(x_df)
clusters = kmeans.predict(x_df)
x_df["Cluster"] = clusters

# use PCA and plot clustering results in 2d
plotdf = pd.DataFrame(np.array(x_df))
plotdf.columns = x_df.columns
pca = PCA(n_components=2)
pcadf = pd.DataFrame(pca.fit_transform(plotdf.drop(["Cluster"], axis=1)))
pcadf.columns = ["PC1_2d", "PC2_2d"]
plotdf = pd.concat([plotdf, pcadf], axis=1, join='inner')
cluster0 = plotdf[plotdf["Cluster"] == 0]
cluster1 = plotdf[plotdf["Cluster"] == 1]
cluster2 = plotdf[plotdf["Cluster"] == 2]
init_notebook_mode(connected=True)
trace1 = go.Scatter(x = cluster0["PC1_2d"], y = cluster0["PC2_2d"], mode = "markers", name = "component 0", marker = dict(color = 'rgba(255, 128, 255, 0.8)'), text = None)
trace2 = go.Scatter(x = cluster1["PC1_2d"], y = cluster1["PC2_2d"], mode = "markers", name = "component 1", marker = dict(color = 'rgba(255, 128, 2, 0.8)'), text = None)
trace3 = go.Scatter(x = cluster2["PC1_2d"], y = cluster2["PC2_2d"], mode = "markers", name = "component 2", marker = dict(color = 'rgba(0, 200, 200, 0.8)'), text = None)
title = "Visualizing Kmean clustering using 2D PCA"
fig = dict(data = [trace1, trace2, trace3], layout = dict(title = title, xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False), yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)))
figure = go.Figure(fig)
figure.show(renderer="colab")

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



clustered well 

In [ ]:
# save data and labels to csv files
sessions.loc[len(sessions.index)] = ['Cluster', 'Cluster'] + list(x_df['Cluster'].T.to_numpy())
sessions = sessions.T


In [ ]:
session1_to_save = pd.DataFrame(np.array(pd.concat([sessions[sessions[16]=='session_id'], sessions[sessions[16]=='1']], ignore_index=True))).T
session1_to_save.rename(columns = {0:'contrast_left', 1:'contrast_right'}, inplace = True)
session2_to_save = pd.DataFrame(np.array(pd.concat([sessions[sessions[16]=='session_id'], sessions[sessions[16]=='2']], ignore_index=True))).T
session2_to_save.rename(columns = {0:'contrast_left', 1:'contrast_right'}, inplace = True)
session3_to_save = pd.DataFrame(np.array(pd.concat([sessions[sessions[16]=='session_id'], sessions[sessions[16]=='3']], ignore_index=True))).T
session3_to_save.rename(columns = {0:'contrast_left', 1:'contrast_right'}, inplace = True)
session4_to_save = pd.DataFrame(np.array(pd.concat([sessions[sessions[16]=='session_id'], sessions[sessions[16]=='4']], ignore_index=True))).T
session4_to_save.rename(columns = {0:'contrast_left', 1:'contrast_right'}, inplace = True)
session5_to_save = pd.DataFrame(np.array(pd.concat([sessions[sessions[16]=='session_id'], sessions[sessions[16]=='5']], ignore_index=True))).T
session5_to_save.rename(columns = {0:'contrast_left', 1:'contrast_right'}, inplace = True)

In [ ]:
# sanity check
assert session1_to_save.shape[1] == session1.shape[1]
assert session2_to_save.shape[1] == session2.shape[1]
assert session3_to_save.shape[1] == session3.shape[1]
assert session4_to_save.shape[1] == session4.shape[1]
assert session5_to_save.shape[1] == session5.shape[1]

In [ ]:
session1_to_save.to_csv("/content/drive/Shareddrives/207/cluster/time_reduced_session_agg1_clustered.csv", index=False)
session2_to_save.to_csv("/content/drive/Shareddrives/207/cluster/time_reduced_session_agg2_clustered.csv", index=False)
session3_to_save.to_csv("/content/drive/Shareddrives/207/cluster/time_reduced_session_agg3_clustered.csv", index=False)
session4_to_save.to_csv("/content/drive/Shareddrives/207/cluster/time_reduced_session_agg4_clustered.csv", index=False)
session5_to_save.to_csv("/content/drive/Shareddrives/207/cluster/time_reduced_session_agg5_clustered.csv", index=False)